# Naive Neural Networks 의 문제점

```
Yesterday, Harry Potter met Hermione Granger
```

- 사람 이름 = 1
- 아니라면 = 0

![](https://i.imgur.com/zPTrFRt.png)

만약 아래 문장이라면 위의 구조로 가능할까?

```
First time Hermione meet Harry and Ron
```

### 기계 번역 문제의 경우, 입력과 출력 시퀀스의 길이가 동일하지 않을수 있다!

# Recurrent Neural Networks (RNN)

![](https://i.imgur.com/owxFJp3.png)

## 장점

- RNN Cell을 여러개 연결하여 유연한 구조를 만든다.
- y^<2> 예측 시 x<2>만 사용하는 것이 아니라, 첫번째 단어로 연산한 정보의 일부(a<1>)를 사용하게 된다.

## 기본적인 RNN 단점

- 앞서 나온 단어들의 정보만을 사용한다.
    - `He said, "Teddy Roosevelt was a great President."`
    - `He said, "Teddy bears are on sale."`
- Gradient vanishing
    - `The cat, which already ate ............, was full`
    - `The cats, which already ate ............, were full`
    - 신경망이 깊어질수록 (문장이 길어질수록) 문장 초반부의 단어가 후반부 단어에 미치는 영향이 적어진다. (가까이 있는 입력 영향을 더 많이 받음)

# RNN 발전형

## RNN

![](https://i.imgur.com/vcUIhTL.png)

## Long Short-Term Memory (LSTM)

![](https://i.imgur.com/VXeCc8x.png)

LSTM은 메모리를 컨베이어 벨트같이 오래 전파하는 특성을 가진다.

## Gated Recurrent Unit (GRU)

![](https://i.imgur.com/fXeGJu6.png)

- 일반적으로 GRU는 LSTM보다 학습 속도가 빠르다.
- 여러 평가에서 GRU는 LSTM과 비슷한 성능을 보인다.
- 데이터 양이 적을 때는 GRU를 사용하는 것이 좋다.

# LSTM 실습

## 자동 완성 모델

In [27]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### 데이터셋

In [28]:
sentences = [
    "i like dog",
    "i love coffee",
    "i hate milk",
    "you like cat",
    "you love milk",
    "you hate coffee"]

### 단어 사전 만들기

In [29]:
word_list = list(set(" ".join(sentences).split()))

word_list

['love', 'you', 'cat', 'coffee', 'milk', 'dog', 'like', 'i', 'hate']

In [30]:
word_dict = {w: i for i, w in enumerate(word_list)}

word_dict

{'love': 0,
 'you': 1,
 'cat': 2,
 'coffee': 3,
 'milk': 4,
 'dog': 5,
 'like': 6,
 'i': 7,
 'hate': 8}

### 데이터 전처리

In [31]:
n_classes = len(word_list)
x_train = []
y_train = []

for sen in sentences:
    words = sen.split(' ')

    x = []
    for w in words[:-1]:
        x.append(word_dict[w])

    y = word_dict[words[-1]]

    x_train.append(np.eye(n_classes)[x])
    y_train.append(np.eye(n_classes)[y])

x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)

print(x_train)
print(y_train)

tensor([[[0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1.]],

        [[0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0.]],

        [[0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1.]]])
tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.]])


### 모델 정의

In [32]:
class TextLSTM(nn.Module):
    def __init__(self):
        super(TextLSTM, self).__init__()

        self.lstm = nn.LSTM(input_size=2, hidden_size=5, batch_first=True)
        self.fc = nn.Linear(5, n_classes)

    def forward(self, hidden_and_cell, x):
        # (batch_size, seq_length, feature_size) (6, 2, 9) -> (6, 9, 2)
        x = x.transpose(1, 2)
        x, (hidden, cell) = self.lstm(x, hidden_and_cell)
        # print(x.shape) # (batch_size, seq_length, hidden_size) (6, 9, 5)

        x = x[:, -1, :]  # (6, 5)
        x = self.fc(x)  # (6, 9)
        return x


model = TextLSTM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

### 학습

In [33]:
batch_size = len(x_train)

for epoch in range(500 + 1):
    hidden = torch.zeros(1, batch_size, 5, requires_grad=True)
    cell = torch.zeros(1, batch_size, 5, requires_grad=True)

    y_pred = model((hidden, cell), x_train)
    loss = criterion(y_pred, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch: {:04d}, cost = {:.6f}'.format(epoch, loss))

Epoch: 0000, cost = 2.092577
Epoch: 0100, cost = 1.286472
Epoch: 0200, cost = 0.479984
Epoch: 0300, cost = 0.094947
Epoch: 0400, cost = 0.028681
Epoch: 0500, cost = 0.016647


### 테스트

In [40]:
input = 'i hate'
# input = 'you like'
# input = 'he liks'

input = input.split(' ')

print(input)

['you', 'like']


In [41]:
x_test = []
for w in input:
    x_test.append(word_dict[w])

x_test = np.eye(n_classes)[x_test]
x_test = torch.FloatTensor([x_test])
x_test

tensor([[[0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0.]]])

In [42]:
hidden = torch.zeros(1, 1, 5, requires_grad=True)
cell = torch.zeros(1, 1, 5, requires_grad=True)

y_pred = model((hidden, cell), x_test)

y_pred

tensor([[-3.6639, -2.9489,  7.2513,  1.8710, -0.5808,  1.1556, -3.0509, -2.9111,
         -2.4495]], grad_fn=<AddmmBackward0>)

In [43]:
torch.softmax(y_pred, dim=1)

tensor([[1.8045e-05, 3.6889e-05, 9.9261e-01, 4.5725e-03, 3.9386e-04, 2.2359e-03,
         3.3313e-05, 3.8308e-05, 6.0780e-05]], grad_fn=<SoftmaxBackward0>)

In [44]:
torch.argmax(torch.softmax(y_pred, dim=1)).cpu().item()

2

In [45]:
word_list[torch.argmax(torch.softmax(y_pred, dim=1)).cpu().item()]

'cat'